In [ ]:
%%file pubsub_topic.sh
#!/bin/bash

export GOOGLE_APPLICATION_CREDENTIALS='/home/meng/work/.GCP_SA/mlee-claritas-bigdata-poc.json'
export REGION='asia-southeast1'
export BUCKET_ID='claritas-bigdata-poc-test'
export TOPIC_ID='test'
export SUBSCRIPTION_ID='store'

# Create a Cloud Storage bucket.
# gsutil mb -l $REGION gs://$BUCKET_ID

# Create a Pub/Sub topic.
gcloud pubsub topics create $TOPIC_ID

# Create a Pub/Sub subscription.
gcloud pubsub subscriptions create $SUBSCRIPTION_ID \
--topic $TOPIC_ID

In [1]:
event_data = """{
    'order_id': '84c538fa-ef6f-11ea-a9c3-3db3abcedca9',
    'timestamp': 20110816,
    'ordered_item': [
        {'item_name': 'Product Trolly', 'item_id': '1001', 'category_name': 'category 1', 'item_price': 189.99, 'item_qty': 1}, 
        {'item_name': 'Product Mystery Box', 'item_id': '1002', 'category_name': 'category 2', 'item_price': 47.0, 'item_qty': 3}, 
        {'item_name': 'Product Gift Bag', 'item_id': '1003', 'category_name': 'category 3', 'item_price': 23.19, 'item_qty': 5}, 
        {'item_name': 'Product Small Block', 'item_id': '1004', 'category_name': 'category 4', 'item_price': 44.99, 'item_qty': 1}, 
        {'item_name': 'Product Cardboard Box', 'item_id': '1005', 'category_name': 'sample category 5', 'item_price': 64.59, 'item_qty': 5}
]}"""

# Data must be a bytestring
event_data.encode('utf-8')

b"{\n    'order_id': '84c538fa-ef6f-11ea-a9c3-3db3abcedca9',\n    'timestamp': 20110816,\n    'ordered_item': [\n        {'item_name': 'Product Trolly', 'item_id': '1001', 'category_name': 'category 1', 'item_price': 189.99, 'item_qty': 1}, \n        {'item_name': 'Product Mystery Box', 'item_id': '1002', 'category_name': 'category 2', 'item_price': 47.0, 'item_qty': 3}, \n        {'item_name': 'Product Gift Bag', 'item_id': '1003', 'category_name': 'category 3', 'item_price': 23.19, 'item_qty': 5}, \n        {'item_name': 'Product Small Block', 'item_id': '1004', 'category_name': 'category 4', 'item_price': 44.99, 'item_qty': 1}, \n        {'item_name': 'Product Cardboard Box', 'item_id': '1005', 'category_name': 'sample category 5', 'item_price': 64.59, 'item_qty': 5}\n]}"

In [6]:
import os
from google.cloud import pubsub_v1

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/meng/work/.GCP_SA/mlee-claritas-bigdata-poc.json'

project_id = 'claritas-bigdata-poc'
topic_id = 'test'

publisher = pubsub_v1.PublisherClient()
# The `topic_path` method creates a fully qualified identifier
# in the form `projects/{project_id}/topics/{topic_id}`
topic_path = publisher.topic_path(project_id, topic_id)

# Data must be a bytestring
# The client returns a future after publish a message
future = publisher.publish(topic_path, event_data.encode('utf-8'))
print(future.result())

print(f"Published messages to {topic_path}.")

2884631842235763
Published messages to projects/claritas-bigdata-poc/topics/test.


In [5]:
import os
from google.cloud import pubsub_v1
from concurrent.futures import TimeoutError

os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/meng/work/.GCP_SA/mlee-claritas-bigdata-poc.json'

project_id = 'claritas-bigdata-poc'
subscription_id = 'store'
# Number of seconds the subscriber should listen for messages
timeout = 5.0

subscriber = pubsub_v1.SubscriberClient()
# The `subscription_path` method creates a fully qualified identifier
# in the form `projects/{project_id}/subscriptions/{subscription_id}`
subscription_path = subscriber.subscription_path(
    project_id, subscription_id)

def callback(message):
    print(f"Received {message}.")
    message.ack()
    
streaming_pull_future = subscriber.subscribe(
    subscription_path, callback=callback)
print(f"Listening for messages on {subscription_path}..\n")

# Wrap subscriber in a `with` block to automatically call close() when done.
with subscriber:
    try:
        # When `timeout` is not set, result() will block indefinitely,
        # unless an exception is encountered first.
        streaming_pull_future.result(timeout=timeout)
    except TimeoutError:
        # Trigger the shutdown.
        streaming_pull_future.cancel()
        # Block until the shutdown is complete.
        streaming_pull_future.result()

Listening for messages on projects/claritas-bigdata-poc/subscriptions/store..

Received Message {
  data: b"{\n    'order_id': '84c538fa-ef6f-11ea-a9c3-3db3abc..."
  ordering_key: ''
  attributes: {}
}.
